In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [2]:
import os
import shutil
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    TrainingArguments,
    Trainer,
    RobertaPreTrainedModel,
    RobertaConfig
)
from transformers.modeling_outputs import SequenceClassifierOutput
from sklearn.model_selection import train_test_split
import evaluate

In [3]:
# --- Configuration ---
MODEL_NAME = "roberta-base"
DATA_FILE = "semcor_train.parquet"
LABEL_MAP_FILE = "label_map.json"
OUTPUT_DIR = "./wsd_roberta_custom"
DRIVE_PATH = "/content/drive/MyDrive/My_WSD_Project"

In [4]:
# Mount Drive
from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"GPU detected: {torch.cuda.get_device_name(0)}")

Using device: cuda
GPU detected: Tesla T4


In [7]:
# --- 2. Load Data & Prepare Mappings ---
print("Loading data...")
df = pd.read_parquet(DATA_FILE)

# Load label mapping
with open(LABEL_MAP_FILE, 'r') as f:
    label2id = json.load(f)

NUM_LABELS = len(label2id)
print(f"Total unique senses (classes): {NUM_LABELS}")

# Split Data
train_df, val_df = train_test_split(df, test_size=0.1, random_state=111)

Loading data...
Total unique senses (classes): 25819


In [8]:
# --- 3. Custom Dataset with Offset Mapping ---
class WSDDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=128):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        sentence = row['sentence']

        # Get exact word boundaries from dataframe
        c_start = row['char_start']
        c_end = row['char_end']

        # Tokenization with offset mapping
        encoding = self.tokenizer(
            sentence,
            truncation=True,
            max_length=self.max_len,
            padding="max_length",
            return_offsets_mapping=True,
            return_tensors="pt"
        )

        # Token search logic
        offsets = encoding['offset_mapping'].squeeze().tolist()
        target_token_idx = 0

        # Find the token that STARTS inside the word boundaries
        for i, (o_start, o_end) in enumerate(offsets):
            if o_start == 0 and o_end == 0: continue # Skip special tokens

            # Logic: if token start matches word start
            if o_start == c_start:
                target_token_idx = i
                break

            # Fallback: if token starts slightly before but covers the word start
            if o_start < c_start and o_end > c_start:
                 target_token_idx = i
                 break

        item = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'target_token_idx': torch.tensor(target_token_idx, dtype=torch.long),
            'labels': torch.tensor(row['label_id'], dtype=torch.long)
        }

        return item

# Initialize Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print("Creating datasets...")
train_dataset = WSDDataset(train_df, tokenizer)
val_dataset = WSDDataset(val_df, tokenizer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Creating datasets...


In [9]:
# --- 4. Custom Model Architecture ---
class RobertaForWSD(RobertaPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.roberta = AutoModel.from_config(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.post_init()

    def forward(self, input_ids=None, attention_mask=None, target_token_idx=None, labels=None, **kwargs):

        # Remove argument that Trainer sends but Model doesn't accept
        kwargs.pop("num_items_in_batch", None)

        # 1. Run RoBERTa
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask, **kwargs)

        sequence_output = outputs.last_hidden_state

        # 2. Extract specific vector
        # Logic remains the same: gather embeddings for the target token index
        batch_size = input_ids.shape[0]
        batch_indices = torch.arange(batch_size, device=input_ids.device)
        target_vectors = sequence_output[batch_indices, target_token_idx]

        # 3. Classification
        target_vectors = self.dropout(target_vectors)
        logits = self.classifier(target_vectors)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

print("Initializing Custom RoBERTa Model...")
model = RobertaForWSD.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS)

Initializing Custom RoBERTa Model...


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForWSD were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# --- 5. Training Setup ---
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    report_to="none",
    dataloader_num_workers=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [11]:
# --- 6. Train ---
print("Starting training RoBERTa...")
trainer.train()

Starting training RoBERTa...


Epoch,Training Loss,Validation Loss,Accuracy
1,3.314100,3.074619,0.593444
2,2.210700,2.397856,0.651746
3,1.673900,2.124056,0.684572
4,1.291300,2.002809,0.705772
5,1.047500,1.984397,0.714146


TrainOutput(global_step=63145, training_loss=2.3121612440087054, metrics={'train_runtime': 8517.4158, 'train_samples_per_second': 118.618, 'train_steps_per_second': 7.414, 'total_flos': 8.186085117845376e+16, 'train_loss': 2.3121612440087054, 'epoch': 5.0})

In [12]:
# --- 7. Save to Drive ---
print("Saving model...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"Copying to Drive: {DRIVE_PATH}")
target_path = os.path.join(DRIVE_PATH, "roberta_wsd_custom")
if os.path.exists(target_path):
    shutil.rmtree(target_path)
shutil.copytree(OUTPUT_DIR, target_path)
shutil.copy(LABEL_MAP_FILE, os.path.join(DRIVE_PATH, "label_map.json"))
print("Done!")

Saving model...
Copying to Drive: /content/drive/MyDrive/My_WSD_Project
Done!
